3325328 Tweets in total

In [25]:
import pandas as pd

from collections import defaultdict
from tqdm import tqdm

from greenspectors.data.twitter_archive_data_loader import TwitterArchiveDataLoader
from greenspectors.env import KEYWORDS, COMPANY_NAMES, SYNONYMS, COMPANY_SPECIFIC_ACTIONS, DATA_PATH

# 1. Load scraped Twitter Archive Data

In [4]:
data_loader = TwitterArchiveDataLoader()

In [6]:
tweet = next(iter(data_loader))
tweet

{'created_at': 'Mon Jun 01 06:53:01 +0000 2020',
 'id': 1267348409419214849,
 'id_str': '1267348409419214849',
 'text': 'RT @jasontoff: I work at Facebook and I am not proud of how we’re showing up. The majority of coworkers I’ve spoken to feel the same way. W…',
 'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
 'truncated': False,
 'in_reply_to_status_id': None,
 'in_reply_to_status_id_str': None,
 'in_reply_to_user_id': None,
 'in_reply_to_user_id_str': None,
 'in_reply_to_screen_name': None,
 'user': {'id': 202007411,
  'id_str': '202007411',
  'name': 'IBookery',
  'screen_name': 'ibookery',
  'location': 'New York, NY',
  'url': None,
  'description': None,
  'translator_type': 'none',
  'protected': False,
  'verified': False,
  'followers_count': 2972,
  'friends_count': 3011,
  'listed_count': 112,
  'favourites_count': 792370,
  'statuses_count': 124683,
  'created_at': 'Wed Oct 13 03:00:29 +0000 2010',
  'utc_offset': None,
  'time_zone': 

# 2. Only keep tweets that contain certain keywords

Filtering approach:

1. Filter for general keywords
2. Filter for Company name + "Sustainability"

In [7]:
def parse_tweet(tweet):
    tweet_data = dict()
    
    retweet = 'retweeted_status' in tweet
    
    tweet_data['id'] = tweet['id']
    tweet_data['created_at'] = tweet['created_at']
    tweet_data['user_id'] = tweet['user']['id']
    tweet_data['username'] = tweet['user']['screen_name']
    tweet_data['name'] = tweet['user']['name']
    tweet_data['place'] = tweet['user']['location']
    if retweet and 'extended_tweet' in tweet['retweeted_status'] and 'full_text' in tweet['retweeted_status']['extended_tweet']:
        tweet_data['tweet'] = tweet['retweeted_status']['extended_tweet']['full_text']
    else:
        tweet_data['tweet'] = tweet['text']
    tweet_data['language'] = tweet['lang']
    tweet_data['mentions'] = tweet['entities']['user_mentions']
    tweet_data['urls'] = tweet['entities']['urls']
    tweet_data['replies_count'] = tweet['reply_count']
    tweet_data['retweet_count'] = tweet['retweet_count']
    tweet_data['likes_count'] = tweet['favorite_count']
    tweet_data['hashtags'] = tweet['entities']['hashtags']
    tweet_data['retweet'] = retweet
    tweet_data['retweet_id'] = tweet['retweeted_status']['id'] if retweet else None
    
    return tweet_data
    
    
    

In [17]:
def process_tweet(tweet):
    tweet_text = tweet['tweet'].lower()
    
    # 1. For every keyword, check whether the tweet belongs to te respective category
    for keyword, keyword_synonyms in KEYWORDS.items():
        keyword_present = False
        for keyword_synonym in [keyword] + keyword_synonyms:
            keyword_parts = keyword_synonym.split(' ')
            all_keyword_parts_present = all([keyword_part.lower() in tweet_text for keyword_part in keyword_parts])
            if all_keyword_parts_present:
                keyword_present = True
                break
        
        if keyword_present:
            keyword_datasets[keyword].append(tweet)
            
    # 2. Filter for Company name + "sustainability"
    for company_name in COMPANY_NAMES:
        company_mentioned = False
        for company_synonym in [company_name] + SYNONYMS[company_name]:
            if company_synonym.lower() in tweet_text:
                company_mentioned = True
                
        if company_mentioned:
            sustainability_mentioned = False
            for sustainability_synonym in ["sustainability"] + KEYWORDS["sustainability"]:
                if sustainability_synonym.lower() in tweet_text:
                    sustainability_mentioned = True
                    break

            if sustainability_mentioned:
                company_sustainability_datasets[company_name].append(tweet)
                break
                
            if company_name in COMPANY_SPECIFIC_ACTIONS:
                action_mentioned = False
                for action in COMPANY_SPECIFIC_ACTIONS[company_name]:
                    if action.lower() in tweet_text:
                        action_mentioned = True
                        break
                
                if action_mentioned:
                    company_actions_datasets[company_name].append(tweet)

# 3. Filter dataset

In [18]:
keyword_datasets = defaultdict(list)
company_sustainability_datasets = defaultdict(list)
company_actions_datasets = defaultdict(list)

for tweet in tqdm(data_loader):
    tweet_data = parse_tweet(tweet)
    process_tweet(tweet_data)
    if tweet_data['retweet']:
        retweet_data = parse_tweet(tweet['retweeted_status'])
        process_tweet(retweet_data)

3325328it [14:19, 3870.80it/s]


In [19]:
for keyword, dataset in keyword_datasets.items():
    print(f"{keyword}: \t {len(dataset)}")

renewable energy: 	 5336
sustainability: 	 4407
carbon: 	 4129
the climate pledge: 	 53
global warming: 	 2013
protect forests: 	 637
fully-electric delivery: 	 34
reduce carbon emission: 	 349
net zero carbon: 	 495
climate pledge fund: 	 19


In [20]:
for company, dataset in company_sustainability_datasets.items():
    print(f"{company}: \t {len(dataset)}")

General Motors: 	 690
Cisco: 	 66
Facebook: 	 573
Ford: 	 807
Wells Fargo: 	 19
Blackrock: 	 104
Amazon: 	 288
Apple: 	 152
Google: 	 103
Visa: 	 77
Samsung: 	 39
Walt Disney: 	 38
Morgan Stanley: 	 7
Exxon Mobile: 	 56
Oracle: 	 26
Microsoft: 	 213
PepsiCo: 	 21
Chevron: 	 142
BASF: 	 3
JetBlue: 	 3
JP Morgan: 	 25
Siemens: 	 26
Unilever: 	 17
Boeing: 	 19
Johnson Johnson: 	 6
Bank of America: 	 9
Mastercard: 	 11
TD Bank: 	 1


In [21]:
for company, dataset in company_actions_datasets.items():
    print(f"{company}: \t {len(dataset)}")

Facebook: 	 1525
Amazon: 	 29
General Motors: 	 364
Ford: 	 2638
Walt Disney: 	 237
Unilever: 	 19
Johnson Johnson: 	 117
Morgan Stanley: 	 11
Microsoft: 	 49
Apple: 	 76
Siemens: 	 5
Exxon Mobile: 	 117
Cisco: 	 7
Wells Fargo: 	 3
Oracle: 	 2
Chevron: 	 119
Mc Donalds: 	 4
PepsiCo: 	 3
Samsung: 	 12
Mastercard: 	 17
JetBlue: 	 3
TD Bank: 	 1
Boeing: 	 1


# 4. Save Company-Action datasets

In [26]:
for company, company_action_dataset in company_actions_datasets.items():
    pd.DataFrame(company_action_dataset).to_csv(f"{DATA_PATH}/Twitter Dataset/Company + Action/{company}.csv", sep='\t', index=False)

# 5. Merge Data with data from twint

In [87]:
def deduplicate(combined_df):
    # Drop ID duplicates and aggregate counts for likes, retweets, etc...
    deduplicated_id = combined_df.drop_duplicates(subset=['id'])
    merged_retweets = deduplicated_id.groupby('tweet', as_index=False)
    
    agg_config = dict.fromkeys(deduplicated_id.columns, 'first')

    del agg_config['tweet']
    agg_config['created_at'] = 'min'
    agg_config['date'] = 'min'
    agg_config['time'] = 'min'
    agg_config['timezone'] = 'min'
    agg_config['replies_count'] = 'sum'
    agg_config['retweet_count'] = 'sum'
    agg_config['likes_count'] = 'sum'
    agg_config['likes_count'] = 'sum'
    
    aggregated = merged_retweets.agg(agg_config)
    
    return aggregated

## 5.1. Keyword datasets

In [28]:
# Some files are named slightly different
KEYWORD_MATCHING = {
    "fully-electric delivery": "fully-electric_delivery_vehicles",
    "protect forests": "protect forests",
    "reduce carbon emission": "reduce_carbon_emissions"
}

In [30]:
SEARCH_TERM_DATA_PATH = f"{DATA_PATH}/TweetsWithGivenSearchTerm_Tashko"

In [86]:
for keyword in KEYWORDS.keys():
    try: 
        df = pd.read_csv(f"{SEARCH_TERM_DATA_PATH}/file_{keyword.replace(' ', '_')}.csv", delimiter="\t")
    except Exception as e:
        df = pd.read_csv(f"{SEARCH_TERM_DATA_PATH}/file_{KEYWORD_MATCHING[keyword]}.csv", delimiter="\t")
        
    if 'retweets_count' in df.columns:
        df = df.rename(columns={'retweets_count': 'retweet_count'})
        
    print(f"{keyword}: {len(df)} + {len(keyword_datasets[keyword])}")
    
    combined_df = pd.concat([df, pd.DataFrame(keyword_datasets[keyword])])
    assert len(combined_df.columns) == 36, "Not 36 columns. Maybe a column name mismatch?"
    
    aggregated = deduplicate(combined_df)
    
    aggregated.to_csv(f"{DATA_PATH}/Twitter Dataset/Keyword/{keyword}.csv", sep='\t', index=False)
    

carbon: 133323 + 4129
sustainability: 47117 + 4407
climate pledge fund: 40 + 19
the climate pledge: 769 + 53
global warming: 24222 + 2013
fully-electric delivery: 22 + 34
net zero carbon: 1973 + 495
protect forests: 1560 + 637
reduce carbon emission: 1315 + 349
renewable energy: 19612 + 5336


## 5.2. Company + Sustainability datasets

In [37]:
COMPANY_MATCHING = {
    'Exxon Mobile': 'Exxon',
    'Mc Donalds': 'McDonalds',
    'Walt Disney': 'WaltDisney',
    'TD Bank': 'TDBank'
}

In [88]:
for company in COMPANY_NAMES:
    try: 
        df = pd.read_csv(f"{SEARCH_TERM_DATA_PATH}/file_{company.replace(' ', '_')}_Sustainability.csv", delimiter="\t")
    except Exception as e:
        df = pd.read_csv(f"{SEARCH_TERM_DATA_PATH}/file_{COMPANY_MATCHING[company]}_Sustainability.csv", delimiter="\t")
        
    if 'retweets_count' in df.columns:
        df = df.rename(columns={'retweets_count': 'retweet_count'})
        
    print(f"{company}: {len(df)} + {len(company_sustainability_datasets[company])}")
    
    combined_df = pd.concat([df, pd.DataFrame(company_sustainability_datasets[keyword])])
    assert len(combined_df.columns) == 36, "Not 36 columns. Maybe a column name mismatch?"
    
    aggregated = deduplicate(combined_df)
    
    aggregated.to_csv(f"{DATA_PATH}/Twitter Dataset/Company + Sustainability/{company}_Sustainability.csv", sep='\t', index=False)
    

Amazon: 164 + 288
Apple: 55 + 152
Bank of America: 80 + 9
BASF: 60 + 3
Blackrock: 60 + 104
Boeing: 9 + 19
Chevron: 21 + 142
Cisco: 153 + 66
Exxon Mobile: 228 + 56
Facebook: 188 + 573
Ford: 60 + 807
General Motors: 3 + 690
Google: 401 + 103
Honeywell: 30 + 0
JetBlue: 60 + 3
Johnson Johnson: 118 + 6
JP Morgan: 24 + 25
Mastercard: 140 + 11
Mc Donalds: 64 + 0
Microsoft: 285 + 213
Morgan Stanley: 40 + 7
Oracle: 94 + 26
PepsiCo: 120 + 21
Samsung: 40 + 39
Siemens: 113 + 26
TD Bank: 20 + 1
Unilever: 24 + 17
Visa: 20 + 77
Walt Disney: 2 + 38
Wells Fargo: 20 + 19
